In [7]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

In [8]:
from pennylane import numpy as np

symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
electrons = 2 

In [9]:
import pennylane as qml

H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is  -0.042072551947439224 * I(0) + 0.1777135822909176 * Z(0) + -0.2427450126094144 * Z(2) + 0.12293330449299361 * (Z(0) @ Z(2)) + 0.1777135822909176 * Z(1) + 0.17059759276836803 * (Z(0) @ Z(1)) + 0.044750084063019925 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.044750084063019925 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.044750084063019925 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.044750084063019925 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.2427450126094144 * Z(3) + 0.16768338855601356 * (Z(0) @ Z(3)) + 0.12293330449299361 * (Z(1) @ Z(3)) + 0.16768338855601356 * (Z(1) @ Z(2)) + 0.1762766139418181 * (Z(2) @ Z(3))


In [10]:
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 0 0]


In [11]:
singles, doubles = qml.qchem.excitations(electrons, qubits, delta_sz=0)
print(singles)
print(doubles)

[[0, 2], [1, 3]]
[[0, 1, 2, 3]]


In [12]:
def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

In [13]:
dev = qml.device("lightning.qubit", wires=qubits)

In [14]:
@qml.qnode(dev, interface="autograd")
def cost_fn(params):
    circuit(params, wires=range(qubits))
    return qml.expval(H)

In [15]:
opt = qml.GradientDescentOptimizer(stepsize=0.8)
np.random.seed(0)  # for reproducibility
theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)
print(theta)

[5.54193389 1.25713095 3.07479606]


In [16]:
max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy = cost_fn(theta)

    conv = np.abs(energy - prev_energy)
    
    if n % 4 == 0:
        print(f"Step = {n}, Energy = {energy:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy:.8f} Ha")
print("\n" f"Optimal value of the circuit parameters = {theta}")


Final value of the ground-state energy = -1.13618832 Ha

Optimal value of the circuit parameters = [3.14350662 3.14087516 2.93185886]


In [17]:
theta

tensor([3.14350662, 3.14087516, 2.93185886], requires_grad=True)

In [36]:
# Build the optimized staate
@qml.qnode(dev)
def optimized_state(params, wires):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)
    return qml.state()

# Get the optimized state
optimized_state_rounded = [np.round(x, 3) for x in optimized_state(theta, wires=range(qubits))]

print(optimized_state_rounded)

[0j, 0j, 0j, (0.105+0j), 0j, 0j, (0.001+0j), 0j, 0j, (-0+0j), 0j, 0j, (-0.995+0j), 0j, 0j, 0j]
